# 데이터 불러오기 및 기본 정보 확인

In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [106]:
# 필요한 라이브러리
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import get_window, welch, spectrogram
from scipy.fft import rfft, rfftfreq
from scipy.stats import entropy
from PIL import Image

# 샘플링 주파수 설정
fs = 100  # Hz, 실험 환경에 맞게 조정

In [289]:
from google.colab import files
uploaded = files.upload()

Saving mpu_data_20250514_203708.csv to mpu_data_20250514_203708.csv


In [301]:
# 파일 불러오기
file_path = 'mpu_data_20250514_203708.csv'  # Colab에 업로드한 파일 경로!!꼭바꿔줘요
df = pd.read_csv(file_path)
# 🔍 시간 부분만 추출: 언더스코어(_) 기준으로 자름
time_str = file_path.split('_')[-1].replace('.csv', '')  # 👉 '201035'

In [302]:
df.head()


,timestamp,accelX,accelY,accelZ,gyroX,gyroY,gyroZ
0,280279,10.70,-6.14,18.61,0.41,8.73,1.85
1,280315,11.79,-4.31,21.63,0.36,8.73,1.73
2,280349,13.15,-5.44,23.90,0.36,8.73,1.59
3,280384,12.87,-4.13,27.22,0.44,8.73,1.45
4,280419,11.29,-4.17,29.97,0.60,8.73,1.36


# 시각화

In [303]:
# 전체 축 시계열 그래프
# 저장할 폴더 경로
save_dir = './plots'

# 폴더가 없으면 생성
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
plt.figure(figsize=(15, 5))
plt.plot(df['accelX'], label='accelX')
plt.plot(df['accelY'], label='accelY')
plt.plot(df['accelZ'], label='accelZ')
plt.title('Time Domain - Accelerometer Data (1-minute)')
plt.xlabel('Sample Index')
plt.ylabel('Acceleration (m/s²)')
plt.legend()
plt.grid(True)
plt.savefig(f'./plots/graph_01.png')
plt.close()

In [304]:
# 이동 평균 계산
window = 14  # 약 0.5초 기준 (샘플링 주기 35ms 가정)
for axis in ['accelX', 'accelY', 'accelZ']:
    df[f'{axis}_ma'] = df[axis].rolling(window=window).mean()

# 시각화
plt.figure(figsize=(15, 5))
plt.plot(df['accelZ'], label='Raw Z', alpha=0.5)
plt.plot(df['accelZ_ma'], label='Moving Average Z', color='red')
plt.title('Z-axis Acceleration with Moving Average')
plt.xlabel('Sample Index')
plt.ylabel('Acceleration (m/s²)')
plt.legend()
plt.grid(True)
plt.savefig(f'./plots/graph_02.png')
plt.close()

# 주파수 분석

In [305]:
# 샘플링 주파수 설정 (Hz)
fs = 1000 / 35  # 실제 환경에 맞게 수정

plt.figure(figsize=(15, 12))

for i, axis in enumerate(['accelX', 'accelY', 'accelZ']):
    signal = df[axis].values
    n = len(signal)
    freq = rfftfreq(n, d=1/fs)
    fft_vals = np.abs(rfft(signal))

    plt.subplot(3, 1, i+1)
    plt.plot(freq, fft_vals)
    plt.xlim(0, 50)
    plt.title(f'FFT of {axis.upper()} (0–50 Hz)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.grid(True)

plt.tight_layout()

plt.savefig(f'./plots/graph_03.png')
plt.close()

In [306]:
plt.figure(figsize=(15, 12))

for i, axis in enumerate(['accelX', 'accelY', 'accelZ']):
    f, psd = welch(df[axis], fs=fs, nperseg=256)

    plt.subplot(3, 1, i+1)
    plt.semilogy(f, psd)
    plt.title(f'Welch PSD of {axis.upper()}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density')
    plt.xlim(0, 50)
    plt.grid(True)

plt.tight_layout()

plt.savefig(f'./plots/graph_04.png')
plt.close()

# 가속도 값 분포

In [307]:
plt.figure(figsize=(15, 4))

for axis in ['accelX', 'accelY', 'accelZ']:
    sns.histplot(df[axis], kde=True, bins=50, label=axis, stat='density')

plt.title('Histogram')
plt.xlabel('Acceleration (m/s²)')
plt.legend()
plt.grid(True)

plt.savefig(f'./plots/graph_05.png')
plt.close()

# 특징

In [308]:
def extract_features_all_axes(data, fs=100):
    features = {}
    for axis in ['accelX', 'accelY', 'accelZ']:
        signal = data[axis].values
        fft_vals = np.abs(rfft(signal))
        f, psd = welch(signal, fs)

        features[f'{axis}_mean'] = np.mean(signal)
        features[f'{axis}_std'] = np.std(signal)
        features[f'{axis}_min'] = np.min(signal)
        features[f'{axis}_max'] = np.max(signal)
        features[f'{axis}_range'] = features[f'{axis}_max'] - features[f'{axis}_min']
        features[f'{axis}_skew'] = pd.Series(signal).skew()
        features[f'{axis}_kurtosis'] = pd.Series(signal).kurtosis()
        features[f'{axis}_fft_peak'] = np.max(fft_vals)
        features[f'{axis}_fft_energy'] = np.sum(fft_vals**2)
        features[f'{axis}_spectral_entropy'] = entropy(fft_vals / np.sum(fft_vals + 1e-12))
        features[f'{axis}_psd_mean_freq'] = np.sum(f * psd) / np.sum(psd)

    return pd.DataFrame([features])

features_df = extract_features_all_axes(df, fs=fs)
print("추출된 특징 요약:")
columns = features_df.columns.tolist()
# 3등분 하기
group1 = columns[:11]
group2 = columns[11:22]
group3 = columns[22:]

# 각각 출력

display(features_df[group1])

display(features_df[group2])

display(features_df[group3])


추출된 특징 요약:


,accelX_mean,accelX_std,accelX_min,accelX_max,accelX_range,accelX_skew,accelX_kurtosis,accelX_fft_peak,accelX_fft_energy,accelX_spectral_entropy,accelX_psd_mean_freq
0,3.627434,6.97958,-10.06,17.99,28.05,-0.033304,-1.424336,6206.54,1.098275e+08,5.420904,1.830827


,accelY_mean,accelY_std,accelY_min,accelY_max,accelY_range,accelY_skew,accelY_kurtosis,accelY_fft_peak,accelY_fft_energy,accelY_spectral_entropy,accelY_psd_mean_freq
0,-5.529725,2.809926,-16.03,6.33,22.36,0.258867,0.775753,9461.36,1.010747e+08,5.938963,2.980694


,accelZ_mean,accelZ_std,accelZ_min,accelZ_max,accelZ_range,accelZ_skew,accelZ_kurtosis,accelZ_fft_peak,accelZ_fft_energy,accelZ_spectral_entropy,accelZ_psd_mean_freq
0,30.474617,7.640345,9.97,45.29,35.32,-0.100908,-0.922734,52142.07,2.804242e+09,4.038112,1.723038


In [309]:


# 열 이름 가져오기
columns = features_df.columns.tolist()

# 3등분
group1 = columns[:11]
group2 = columns[11:22]
group3 = columns[22:]

groups = [group1, group2, group3]

# 테이블 이미지로 저장 (각 그룹별로)
for i, group in enumerate(groups, start=1):
    fig, ax = plt.subplots(figsize=(15, 1))
    ax.axis('off')
    tbl = ax.table(cellText=features_df[group].values,
                   colLabels=features_df[group].columns,
                   loc='center')
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(7)
    plt.tight_layout()
    plt.savefig(f'./plots/features_table_part_{i:02d}.png')
    plt.close()


In [310]:
os.listdir(save_dir)


['features_table_part_03.png',
 'features_table_part_02.png',
 'graph_04.png',
 'graph_05.png',
 'features_table_part_01.png',
 'graph_03.png',
 'graph_01.png',
 'graph_02.png']

In [311]:
from PIL import Image
import os

save_dir = './plots'

# 이미지 파일 리스트 가져오기
image_files = sorted([
    os.path.join(save_dir, f)
    for f in os.listdir(save_dir)
    if f.lower().endswith('.png')
])

# 이미지 없으면 에러 출력
if not image_files:
    raise FileNotFoundError("⚠️ 저장된 PNG 이미지가 plots 폴더에 없습니다.")

# 이미지 열기
images = [Image.open(img) for img in image_files]

# 최대 너비와 전체 높이 계산
max_width = max(img.width for img in images)
total_height = sum(img.height for img in images)

# 새 이미지 생성
combined_img = Image.new('RGB', (max_width, total_height), color=(255, 255, 255))

# 이미지 하나씩 붙이기
y_offset = 0
for img in images:
    combined_img.paste(img, (0, y_offset))
    y_offset += img.height

# 최종 저장
save_name = f'60psi_1_{time_str}.png'
combined_img.save(save_name)
print(f" 모든 이미지가 '{save_name}' 파일로 저장되었습니다.")
files.download(save_name)


 모든 이미지가 '60psi_1_203708.png' 파일로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [312]:
#이미지 모아서 병합한거엿는데 모은 이미지 삭제하는거임 다음 파일 전에 꼭 진행부탁해요!
import shutil
shutil.rmtree('./plots')  # plots 폴더 전체 삭제
